# Backtest 2020

## What is the backtest?

The backtest analyzes data from a past period of time to determine the ideal values of seven different metrics (discussed later) for each sector.  This version of the backtest uses the jupyter notebook format to describe each part, as well as to increase efficiency.

###  
### Library Imports

- pandas - used to manipulate data within different data structures
- datetime - used to get today's date and current time
- time - used for calculating runtime
- numpy - supports mathematical functions for large volume data structures
- product - creates a cartesian product

In [1]:
from pandas import *
import pandas as pd
from datetime import *
import time
import numpy as np
from itertools import product
from IPython.display import display

### Effiency and timing

In the past, it has taken dozens of hours to complete the backtests.  This is costly in terms of time and electricity (which should be considered as a sustainability-oriented organization).  The block below assigns the current time to a variable so that we can later determine runtime.

We also establish our sectors and metrics in lists.

In [2]:
metrics = [ 'PE_RATIO', 'PX_TO_BOOK_RATIO', 'TRAIL_12M_EPS', 'TOT_DEBT_TO_TOT_EQY', \
               'PX_TO_FREE_CASH_FLOW', 'RETURN_COM_EQY', 'RETURN_ON_ASSET' ]

#These are used in creating and manipulating dataframes
dfTests = ['peTest','pbTest','epsTest','deTest','fcfTest','roeTest','roaTest']
dfScores = ['peScore','pbScore','epsScore','deScore','fcfScore','roeScore','roaScore']

lvhs_metrics = [ 'PE_RATIO', 'TOT_DEBT_TO_TOT_EQY', 'PX_TO_BOOK_RATIO' ] # Low-value high score metrics
hvhs_metrics = [ 'TRAIL_12M_EPS', 'PRICE_TO_FCF', 'RETURN_COM_EQY', 'RETURN_ON_ASSET' ] # High value high score metrics

pd.options.mode.chained_assignment = None #allow chained assignment

#  
#   
# Step 1: Data input and Setup, Organic Range Generation

The basis of our backtest are frames - sets of each metric, each containing a different combination of values.  In order to determine what these values are, we have to come up with a range for each metric, for each sector.

### Data Import

We first import our data file for range generation -- the last 10 years of data for each company.  The data is imported to a **dataframe** (DF), a data structure which can be thought of as an excel sheet manipulable through python.  The dataframe is then sorted by date.

The line reading "totalDF.dropna..." drops rows where the date is empty.  We then establish a start date (generally five years prior to the test being run), and establish the date format as "ordinal."  This ensures that our date formatting is compatible with that of our code.  Finally, we cut the data based on when our start date is, so that we are not using more data than is necessary and doing an excessive amount of computing.

In [24]:
totalDF = read_csv("Backtest VALUES 2020.csv",index_col='DATE',parse_dates=True)

totalDF = totalDF.sort_index()

#drop any rows without a date
totalDF.dropna(axis='index',how='any',inplace=True)
display(totalDF)

#Define starting date to begin df with
startDate = to_datetime('2015-09-01')
startDate=startDate.toordinal()

#Cut totalDF based on DateTime range
totalDF['ordinalTime']=totalDF.index
totalDF['ordinalTime'] = totalDF['ordinalTime'].apply(date.toordinal)
totalDF = totalDF.loc[totalDF['ordinalTime']>=startDate]

#Create set of all of the dates contained in totalDF
datesList = sorted(set(totalDF.index))
print(datesList)
#sectors = sorted(set(totalDF.GICS_SECTOR_NAME.unique()))
sectors = totalDF.GICS_SECTOR_NAME.unique()
print(sectors)


,GICS_SECTOR_NAME,TICKER,Q_Open_Price,PX_LAST,RETURN,PE_RATIO,PX_TO_BOOK_RATIO,TRAIL_12M_EPS,TOT_DEBT_TO_TOT_EQY,PX_TO_FREE_CASH_FLOW,RETURN_COM_EQY,RETURN_ON_ASSET,CUR_MKT_CAP,ADJUSTED_BETA
DATE,,,,,,,,,,,,,,
2009-03-02,Health Care,A US Equity,9.7252,9.0673,-0.067649,5.2021,1.2167,1.7430,85.7254,5.3539,22.1797,8.7135,4463.1154,0.9889
2009-03-02,Information Technology,NTAP US Equity,13.1900,12.9000,-0.021986,17.1857,2.7921,0.7506,82.8209,5.1796,5.3937,1.9171,4259.1003,1.0664
2009-03-02,Industrials,NSC US Equity,30.7500,29.5600,-0.038699,6.5110,1.1269,4.5400,69.3973,9.5151,17.7511,6.5445,10832.5808,0.9986
2009-03-02,Energy,NOV US Equity,23.3235,21.6021,-0.073805,4.3729,0.7140,4.9400,6.8684,4.5710,20.2395,11.6213,10003.9009,1.4922
2009-03-02,Industrials,NOC US Equity,33.2300,32.1590,-0.032230,17.2715,0.8823,1.8620,33.0872,4.2268,-8.5250,-3.9704,11598.5491,0.7823
2009-03-02,Consumer Discretionary,NKE US Equity,10.2075,9.9400,-0.026206,9.9716,2.3457,0.9968,9.7539,13.1948,20.6749,13.6068,19246.7658,0.9363
2009-03-02,Consumer Discretionary,NFLX US Equity,5.0570,4.9070,-0.029662,27.0472,5.8243,0.1814,11.2745,8.6717,21.3718,12.8283,2015.8057,0.9365
2009-03-02,Financials,NDAQ US Equity,20.2800,19.8600,-0.020710,9.7353,0.9355,2.0400,58.6568,20.2488,9.6700,3.9920,4010.2542,1.2896
2009-03-02,Health Care,MYL US Equity,12.1600,11.6200,-0.044408,42.6948,1.2845,0.2722,187.8523,16.1199,-10.4073,-1.6681,3540.6663,0.9454


[Timestamp('2015-09-01 00:00:00'), Timestamp('2015-12-01 00:00:00'), Timestamp('2016-03-01 00:00:00'), Timestamp('2016-06-01 00:00:00'), Timestamp('2016-09-01 00:00:00'), Timestamp('2016-12-01 00:00:00'), Timestamp('2017-03-01 00:00:00'), Timestamp('2017-06-01 00:00:00'), Timestamp('2017-09-01 00:00:00'), Timestamp('2017-12-01 00:00:00'), Timestamp('2018-03-01 00:00:00'), Timestamp('2018-06-01 00:00:00'), Timestamp('2018-09-04 00:00:00'), Timestamp('2018-12-03 00:00:00'), Timestamp('2019-03-01 00:00:00'), Timestamp('2019-06-03 00:00:00'), Timestamp('2019-12-02 00:00:00'), Timestamp('2020-03-02 00:00:00'), Timestamp('2020-06-01 00:00:00'), Timestamp('2020-09-01 00:00:00'), Timestamp('2020-09-03 00:00:00')]
['Consumer Discretionary' 'Materials' 'Financials' 'Industrials' 'Energy'
 'Information Technology' 'Consumer Staples' 'Health Care' 'Real Estate'
 'Telecommunication Services' 'Utilities']


### getRanges method

The getRanges method returns a list of ranges, one for each metric, for that specific sector.  A range has a low value, high value, and step value (incrementation value).  While this code is visually unappealing, it's relatively practical. 

It is worth noting that we take ranges from the 25th percentile to the 75th percentile (the middle section of a normal distribution.  If you're unsure what a normal distribution looks like and why we chose these percentages, I suggest researching these so that you have a better understanding of the statistics behind this).  The companies in the top 25% for a metric may be outliers or overvalued based on that number, and the companies in the lower 25% may be underperforming.  

It is also worth noting the step values.  These are generally safe bets for typical output to create a reasonable number of frames.  Remember, runtime is **_exponential_**.*  One of our current tasks for improving the model is to develop an algorithm to calculate the step value; for example if the range starts at 20 and goes up to 120, a step of 10 has 12 potential frame values, while a step value of 5 has 24.  The fewer frame values we have, the more quickly the code will run; too large of a step value, and we may be distorting the accuracy of the backtest.

First, we have to import the data again as an excel sheet.  This is due to some conflicts with "None" values as read by a CSV file.  We then sort the data by date and group the rows by sector.

We then define a function which gets the appropriate data for each metric and returns the ranges as a tuple.

In [25]:
#Using the same CSV file creates issues with the slice(None) function
#Try to fix this .xlsx issue wher it doesn't allow slicing
totalDF_ranges = pd.read_excel("Backtest VALUES 2020.xlsx", index_col=[0,1])
totalDF_ranges = totalDF_ranges.sort_index()
totalDF_ranges.groupby('GICS_SECTOR_NAME').nunique()

#Unlike typical range functions, the step value must be listed, even if it is 1
#This function calculates and returns the normal range given the sector, metric, and step
def getNormal(metric, step):
    return range(int(round(np.percentile(totalDF_ranges.loc[(slice(None), sector), metric].dropna(), 34))), \
                     int(round(np.percentile(totalDF_ranges.loc[(slice(None), sector), metric].dropna(), 68))), step)

def getRanges():
    #define these values as global so that we can use them in other functions
    global peRange, pbRange, epsRange, deRange, fcfRange, roeRange, roaRange
    
    peRange =  getNormal('PE_RATIO', 2)
    pbRange =  getNormal('PX_TO_BOOK_RATIO', 1)
    epsRange =  getNormal('TRAIL_12M_EPS', 2)
    deRange =  getNormal('TOT_DEBT_TO_TOT_EQY', 10)
    fcfRange =  getNormal('TOT_DEBT_TO_TOT_EQY', 5)
    roeRange =  getNormal('RETURN_COM_EQY', 2)
    roaRange =  getNormal('RETURN_ON_ASSET', 2)

#  
#  
# Step 2: Function Definitions

### Output functions

The backtest prints out three sheets as output.

SectorNameIndPortPeriodDate is a sheet showing hypothetical groupings of companies, per quarter, which fit each frame.  It gives the data for each of these companies during that quarter.  This is the largest sheet; the number of frames times the number of quarters = the number of rows in this spreadsheet. 

SectorNameIndividualPortfoliosDate takes the same data and displays it without showing each individual company in each test.  It shows the results of how each frame performed each quarter.

SectorNameTotalPortfolioDate prints out the ultimate performance of each frame's average performance over time.  This is ordered by Treynor ratio, so the best frame (highest Treynor ratio) will be the first line.

In [26]:
#Establishing the date now prevents an error where if the backtest is running and the time turns 
#midnight, it is unable to write to the correct files because the date has changed
now = time.strftime("d%dm%my%Y")
#Function to output every indivdual portfolio inside each period
def outputIndPeriod(portfolioDF, date):
        portfolioDF.to_csv("results/"+sector+"IndPortPeriod"+now+".csv", mode='a')
        
#output each portfolio with dates
def outputIndPort(portSummaryDF, frame):
    portSummaryDF.to_csv("results/"+sector+"_PortfolioAvgsPerPeriod"+now+".csv", \
                  mode='a',index_label=("T"+str(frame[0])+str(frame[1])+str(frame[2])+str(frame[3])\
                        +str(frame[4])+str(frame[5])+str(frame[6])))

#Output total portfolio with parameters
def outputTotalPort(dfTotalPort, frame):
    dfTotalPort.to_csv("results/"+sector+"TotalPortfolio"+now+".csv", mode='w',index_label=("T"+"Portfolio"))
    

### Calculate Portfolios

To understand this function, it's also important to understand the Cartesian product and the prod function.

Let's look at the function definition.  The datesList comes from way down below; we basically create a list of all of the dates in our data.  "Params" comes from the prod function; it contains a frame, which is a list containing a hypothetical value for each metric.  Sect is simply the name of the sector.  Params, or the frame values, only changes when the calculatePortLoop is called from the loop in the prod() function.

The for loop is based around our dates list.  For each date (which equates each quarter), we want to do the following:

1. Take all data pertaining to that date/quarter and create a new dataframe (df.
2. Format the dataframe based on our metrics and insert the frame values.
3. Score whether or not the actual data is at least as good as the frame value.  This is listed as a TRUE or FALSE value.
4. Calculate the HSF score. This counts the number of TRUE values from the last step and adds the adjusted beta, which has been divided by 1000 so as not to disrupt the innate value of the HSF score.
5. An identifying tag is established for each frame.  The frame value is used to generate this tag.  This is sort of like an SKU code used in retail.
6. The dataframe is sorted by highest HSF Score and the top 5 companies are selected.  These are added to the SectorNameIndPortPeriodDate.csv file.


7. The tested frame values are added to the portSummaryDF dataframe, which is established much further below.  The highest HSF score, as well as the average return and average adjusted beta, are also added to this dataframe.  
8. The average return and beta are then used to calculate a Treynor ratio; basically, this represents risk vs. return for those top 5 HSF scoring companies for the frame and date. 
9. The frame is also added, formatted as a list like (0, 1, 2, etc).  portSummaryDF is then returned (to the prod() function).

In [27]:
#This function creates a unique identifier using frame values
def createIdentifier(frame):
    name = "T"
    for p in frame:
        name+=str(p)
    return name

In [32]:
#Function to loop through every date and output an individual portfolio
def calculatePortLoop(dateList, frame, identity):
    
    #Go through the dates list.  frame will remain the same through the for loop, and only changes when this function
    #is called again from the prod() function.
    for date in datesList:
        
        #create a dataframe based on the date
        portfolioDF = sectorDF.loc[sectorDF.index==date]
        
        #Adds the values of the frame to the dataframe
        for x in range(7):
            portfolioDF[dfTests[x]] = frame[x]
        
        #Scores whether or not the value for each metric is at least as good as the frame value
        for x in range(7):
            if metrics[x] in lvhs_metrics:
                portfolioDF[dfScores[x]] = portfolioDF[metrics[x]] < frame[x]
            else:
                portfolioDF[dfScores[x]] = portfolioDF[metrics[x]] > frame[x]
        
        #Re-calculate beta score so that we can add it to the HSF score
        portfolioDF['betaScore']=1/(portfolioDF['ADJUSTED_BETA']*1000)
        #Calculate HSF score
        portfolioDF['hsfScore'] = None
        for x in range(7):
            portfolioDF['hsfScore']=testDF['hsfScore']+testDF[dfScores[x]].astype(int)
            
        #Creates an identifier for each frame based on the frame values
        portfolioDF['ind'] = identity
        
        #Sorts the dataframe by highest HSF score
        portfolioDF.sort_values(by='hsfScore',ascending=False,inplace=True)
        #Add only the 5 companies with the highest HSF score to the dataframe.
        portfolioDF=portfolioDF.iloc[0:5]
        
        #Print to the SectorNameIndPortPeriodDate.csv file.  This happens each time we go through the loop
        if iperiods:
            outputIndPeriod(portfolioDF, date)
        
        #portSummaryDF is for SectorName_PortfolioAvgsPerPeriod.csv
        for t in dfTests:
            portSummaryDF.loc[portSummaryDF.index==date, t] = portfolioDF.ix[0, t]
        
        portSummaryDF.loc[portSummaryDF.index==date,'maxHSFScore'] = portfolioDF['hsfScore'].max()
        #Finds the average return and adjusted beta of the five companies.
        portSummaryDF.loc[portSummaryDF.index==date,'return'] = (portfolioDF['RETURN'].mean())
        #Average beta is calculated.  It's possible to have the average come out to zero, which gives us an error
        #later when calculating the Treynor ratio.  I.E., we add a very small amount to prevent this issue with minimal
        #statistical impact
        portSummaryDF.loc[portSummaryDF.index==date,'beta'] = (portfolioDF['ADJUSTED_BETA'].mean()+0.0000001)

        
    #Calculate a Treynor ratio for the portSummaryDF dataframe
    portSummaryDF['treynor']=(portSummaryDF['return']-TBill3Mth)/portSummaryDF['beta']
    #Adds the frame in list formatting to the portSummaryDF dataframe
    portSummaryDF['ind'] = identity

    return portSummaryDF

### Cartesian Product

Now, we'll look at the Cartesian product.  This is where things get slightly more mathematical, and where we can fully grasp the importance of efficiency.  

The only argument of the prod() function is the sector name, and this is just to pass to other functions which need that name to create our .csv files.

This for loop is the heart of our backtest.  Keep in mind that in python, a function can be called in the for loop if it returns a list which can be traversed.  In this case, that function is the product() function; more specifically, this creates a Cartesian product.  From thefreedictionary.com:

    A cartesian product is "the set of all ordered pairs of members of two given sets. The product A × B is the set of all pairs <a, b> where a is a member of A and b is a member of B."
    
Think of it like this.  Say we have two sets of numbers, A and B.  Say that A has the values (1, 2, 3) and B has the values (4, 5, 6).  A cartesian product between A and B would produce  all of these possible sets:

(1, 4)
(1, 5)
(1, 6)
(2, 4)
(2, 5)
(2, 6)
(3, 4)
(3, 5)
(3, 6)

Each number from A is combined with each number from B.  Note that the number of possible sets is equal to the number of values in A times the number of values in B: 3 x 3 = 9.  If B had four numbers, there would be 3 x 4 = 12 possible combinations, and so forth and so on.  Now consider how this works with our ranges; an example for the number of values in each range is below:

    peRange = 4 possible values

    pbRange = 5 possible values

    epsRange = 6 possible values

    deRange = 4 **

    fcfRange = 4 **

    roeRange = 4 **

    roaRange = 3 **


4 x 5 x 6 x 4 x 4 x 4 x 3 = 23,040 possible combinations.  This is the number of frames that the for loop would have to run through.  Say that roaRange changed from 3 to 4; this would give us 30,720 combinations.  As you can see, the ranges create an exponentially large set of combinations; this is why the backtest takes so long to run, and why efficiency is so important.

So how does the rest of the function work?  

1. The for loop takes a single frame from the cartesian product and calls the calculatePortLoop function from above.  As we know, this adds data to the SectorNameIndPortPeriodDate.csv file and to the portSummaryDF dataframe.
2. dfTotalPort is the dataframe to show the final results.  We calculate the average return for the frame over time as well as the average beta, and insert these into the dfTotalPort dataframe.
3. The frame values are copied into dfTotalPort.
4. The SectorNameIndividualPortfoliosDate.csv is printed to using the portSummaryDF dataframe.
5. The loop continues to do this for each frame.
6. After the loop has finished, the total Treynor for each frame is calculated using the average beta and average return for each frame over time, as well as risk-free rate.  This data is added to the dfTotalPort dataframe.
7. The dfTotalPort dataframe is then sorted by Treynor ratio, with the highest Treynors appearing at the top. Keep in mind that Treynor ratio shows return vs. risk.  Higher Treynor values are considered better, so the first Treynor listed on this sheet will be for the best frame.
8. Similar to above, identifying tags are again generated for each frame.  They match the format as in the last function.
9.  outputTotalPort is called, allowing for us to print out the SectorNameTotalPortfolioDate.csv file.

In [33]:
def prod():
    
    #df, portSummaryDF, dfTotalPort
    for frame in product(peRange, pbRange, epsRange, deRange, fcfRange, roeRange, roaRange):
        identifier = createIdentifier(frame)
        #Calculate portfolios
        calculatePortLoop(datesList, frame, identifier)
        #Average return for the frame over time
        dfTotalPort.loc[dfTotalPort['ind']==frame,'meanReturn'] = portSummaryDF['return'].mean()
        #Average Beta for the frame over time
        dfTotalPort.loc[dfTotalPort['ind']==frame,'meanBeta'] = portSummaryDF['beta'].mean()
        #Repeats the frame values into the dataframe
        for x in range(7):
            dfTotalPort.loc[dfTotalPort['ind']==frame, dfTests[x]] = frame[x]
        
        #print(portSummaryDF)
        if iports:
            outputIndPort(portSummaryDF, frame)
        
    #Calculate Treynor
    dfTotalPort['totalTreynor']=(dfTotalPort['meanReturn']-dfTotalPort['tBill3Mth'])/dfTotalPort['meanBeta']
    #Sort by Treynor
    dfTotalPort.sort_values('totalTreynor',ascending=False, inplace=True)
    #Creates an identifying string using the values of the frame
    dfTotalPort.set_index("T"+dfTotalPort['peTest'].map(str) + dfTotalPort['pbTest'].map(str) +\
                      dfTotalPort['epsTest'].map(str) + dfTotalPort['deTest'].map(str) +\
                      dfTotalPort['fcfTest'].map(str) + dfTotalPort['roeTest'].map(str) +\
                      dfTotalPort['roaTest'].map(str), inplace = True)
    
    #Print to CSV file
    outputTotalPort(dfTotalPort, frame)

#  
#  
# Step 3: Naming sector, Producing Output Per Sector

There are two functions below.  The first runs the backtest for a single sector.  The second runs the backtest for every sector.

We first name a sector.  The ranges for this sector are then generated by calling the getRanges(sector) function, which returns a tuple containing each range.

A new dataframe, sectorDF, is created.  It takes all of the data from totalDF that pertains to companies in the defined sector.

The output dataframes are initialized as empty.  portSummaryDF will be used for the SectorNameIndividualPortfoliosDate.csv file, and dfTotalPort will be used for the SectorNameTotalPortfolioDate.csv file, as explained above.

Finally, the prod() function is called.  This will run the Cartesian product and all of the output functions, printing all of the necessary data to our csv files.

In [34]:
def runSingleSector(sectorChosen, tbill, libor, IndPortfolios, IndPeriods):

    global sector
    sector = sectorChosen
    
    getRanges()

    #Define sectorDF for use in other functions
    global sectorDF
    sectorDF = totalDF.loc[totalDF['GICS_SECTOR_NAME']==sector]
    
    #Define the risk-free rates
    global TBill3Mth
    TBill3Mth = tbill/100
    global Libor3Mth
    Libor3Mth = libor/100
    
    #Define whether or not we want excel sheets containing individual portfolio models and periodic models
    global iports
    iports = IndPortfolios
    global iperiods
    iperiods = IndPeriods
    
    #Create new Output DF for portfolios for dates
    global portSummaryDF
    portSummaryDF = pd.DataFrame([],index=datesList)
    portSummaryDF['return']=None
    portSummaryDF['beta']=None
    portSummaryDF['tBill3Mth']=TBill3Mth
    portSummaryDF['treynor']=None
    for x in range(7):
            portSummaryDF[dfTests[x]] = None
    portSummaryDF['maxHSFScore']=None

    #Create new Output DF for total portfolios
    global dfTotalPort
    dfTotalPort = pd.DataFrame([])
    #assign product function to local variable
    dfTotalPort['ind']=list(product(peRange,pbRange,epsRange,deRange,fcfRange,roeRange,roaRange))
    dfTotalPort['meanReturn']=None
    dfTotalPort['meanBeta']=None
    dfTotalPort['tBill3Mth']=TBill3Mth
    dfTotalPort['totalTreynor']=None
    for x in range(7):
            dfTotalPort[dfTests[x]] = None

    #Call the prod() function
    prod()

#  
#  
# Function Calls and Runtime

The following calls the desired function and calculates runtime.  In order to call the function, we have to put in our Risk-Free rates (T-Bill 3-month average and Libor 3-month average) in order to run the function.  These change regularly, so they should be checked each time the model is run.

In [35]:
start = time.time()

runSingleSector('Telecommunication Services', 1.52, 1.81, False, False)

#The two lines below would allow us to run all sectors at once
#for s in sectors:
#    runSingleSector(s, tbill, libor, False, False)

stop = time.time()

print('Time: ', stop - start)  

NameError: name 'testDF' is not defined